In [ ]:
import importlib
import functions.core_functions as core_functions
import dask
import dask.dataframe as dd
from dask.dataframe.utils import assert_eq
# import dask_cudf as dc
import pandas as pd
import numpy as np
import pandas_gbq

dask.config.set({"dataframe.backend": "cudf"})



importlib.reload(core_functions)


resp = {}
resp = core_functions.initialize_clients()

config = resp.get('config')
bigquery_client = resp.get('clients').get('bigquery_client')
storage_client = resp.get('clients').get('storage_client')
sf_client = resp.get('clients').get('sf_client')
veil_billing = resp.get('config').get('veil_billing')
veil_vars = resp.get('config').get('veil_billing').get('vars')
print(veil_billing)



In [ ]:
# config




In [ ]:


target_table = 'encodings'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
encodings_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'encoders'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
encoders_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'encoder_groups'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
encoder_groups_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)


target_table = 'formats'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
formats_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'customers'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
customers_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'profiles'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
profiles_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'aeismaps'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
aeismaps_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)


target_table = 'sfdc_bvs_customer__c_obj'
fetch_sql = f"""
SELECT * from {veil_billing.get('billing_project_id')}.{veil_billing.get('billing_dataset_id')}.{target_table}"""
sfdc_bvs_customer_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'sfdc_bvs_format__c_obj'
fetch_sql = f"""
SELECT * from {veil_billing.get('billing_project_id')}.{veil_billing.get('billing_dataset_id')}.{target_table}"""
sfdc_bvs_format_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'sfdc_account_obj'
fetch_sql = f"""
SELECT * from {veil_billing.get('billing_project_id')}.{veil_billing.get('billing_dataset_id')}.{target_table}"""
sfdc_account_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'sfdc_advertiser__c_obj'
fetch_sql = f"""
SELECT * from {veil_billing.get('billing_project_id')}.{veil_billing.get('billing_dataset_id')}.{target_table}"""
sfdc_advertiser_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'sfdc_rate_card__c_obj'
fetch_sql = f"""
SELECT * from {veil_billing.get('billing_project_id')}.{veil_billing.get('billing_dataset_id')}.{target_table}"""
sfdc_rate_card_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

# target_table = 'sfdc_rate_card__c_obj'
# fetch_sql = f"""
# SELECT * from {veil_billing.get('billing_project_id')}.{veil_billing.get('billing_dataset_id')}.{target_table}"""
# sfdc_rate_card_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)



In [ ]:
attributes_list = ['product_code', 'product_name', 'donovan_agency_product_code', 'description', 'isci', 'project_name', 'advertiser', 'client_code',
                   'cable_estimate', 'spot_estimate', 'campaign', 'audience', 'audience2', 'category', 'comercial_id', 'contour_id', 'creative_offer',
                   'donovan_agency_advertiser_code', 'donovan_agency_estimate_code', 'eid', 'group', 'hd_sd', 'id', 'length', 'lob', 'media_type',
                   'message', 'misc', 'module_code', 'offer', 'offer_2', 'phone_number', 'quality', 'revision', 'show_name', 'slug', 'sport_id',
                   'sport_show_sub_category', 'spot_name', 'tag', 'text', 'title', 'veil_id', 'version_name', 'year']
for attr in attributes_list:
    encodings_df[attr] = encodings_df['attributes'].apply(lambda x: x.get(attr))
# encodings_df['product_code'] = encodings_df['attributes'].apply(lambda x: x.get('product_code'))

# Define conditions
# Ensure 'description' column does not contain None values
encodings_df['description'] = encodings_df['description'].fillna('')

# Define conditions
conditions = [
    encodings_df['product_code'].notnull(),
    encodings_df['product_code'].isnull() & encodings_df['product_name'].notnull(),
    encodings_df['product_code'].isnull() & encodings_df['product_name'].isnull() & encodings_df['donovan_agency_product_code'].notnull(),
    encodings_df['product_code'].isnull() & encodings_df['product_name'].isnull() & encodings_df['donovan_agency_product_code'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & ~encodings_df['description'].str.startswith(('TV', 'RA')),
    encodings_df['product_code'].isnull() & encodings_df['product_name'].isnull() & encodings_df['donovan_agency_product_code'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & encodings_df['description'].str.startswith(('TV', 'RA'))
]

# Define corresponding values
choices = [
    encodings_df['product_code'],
    encodings_df['product_name'],
    encodings_df['donovan_agency_product_code'],
    encodings_df['description'].str[26:30].str.strip(),
    encodings_df['description'].str[6:10].str.strip()
]

# Apply conditions and choices to create the new column
encodings_df['product_code'] = np.select(conditions, choices, default=None)



# Define conditions
conditions = [
    encodings_df['isci'].notnull(),
    encodings_df['isci'].isnull() & encodings_df['project_name'].notnull(),
    encodings_df['isci'].isnull() & encodings_df['project_name'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & ~encodings_df['description'].str.startswith(('TV', 'RA')),
    encodings_df['isci'].isnull() & encodings_df['project_name'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & encodings_df['description'].str.startswith(('TV', 'RA'))
]

# Define corresponding values
choices = [
    encodings_df['isci'],
    encodings_df['project_name'],
    encodings_df['description'].str[8:18].str.strip(),
    encodings_df['description'].str[18:38].str.strip()
]

# Apply conditions and choices to create the new column
encodings_df['isci'] = np.select(conditions, choices, default=None)


# Define conditions
conditions = [
    encodings_df['advertiser'].notnull(),
    encodings_df['advertiser'].isnull() & encodings_df['client_code'].notnull(),
    encodings_df['advertiser'].isnull() & encodings_df['client_code'].isnull() & encodings_df['donovan_agency_advertiser_code'].notnull(),
    encodings_df['advertiser'].isnull() & encodings_df['project_name'].isnull() & encodings_df['donovan_agency_advertiser_code'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & ~encodings_df['description'].str.startswith(('TV', 'RA')),
    encodings_df['advertiser'].isnull() & encodings_df['project_name'].isnull() & encodings_df['donovan_agency_advertiser_code'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & encodings_df['description'].str.startswith(('TV', 'RA'))
]

# Define corresponding values
choices = [
    encodings_df['advertiser'],
    encodings_df['client_code'],
    encodings_df['donovan_agency_advertiser_code'],
    encodings_df['description'].str[22:26].str.strip(),
    encodings_df['description'].str[2:6].str.strip()
]

# Apply conditions and choices to create the new column
encodings_df['advertiser'] = np.select(conditions, choices, default=None)
encodings_df

In [ ]:
sfdc_adv_account_cols = [
 'recordtypeid_adv',
 'createddate_adv',
 'createdbyid_adv',
 'lastmodifieddate_adv',
 'lastmodifiedbyid_adv',
 'systemmodstamp_adv',
 'lastactivitydate_adv',
 'lastvieweddate_adv',
 'lastreferenceddate_adv',
 'billing_frequency',
 'notes',
 'po_number',
 'po_required',
 'term_date_end',
 'term_date_start',
 'active_po',
 'product_code',
 'status',
 'advertiser_name_location',
 'adv_acct',
 'format_code',
 'invoice_format',
 'cloned_advertiser',
 'bvs_format_id',
 'match_type',
 'purchase_order_record',
 'radio_rate_card',
 'radio_advertiser',
 'product_code_list',
 'encoding_advertiser',
 'encoding_module_code',
 'encoding_product_code',
 'encoder_group',
 'encoding_format',
 'bvs_customer_id',
 'encoding_format_id',
 'encoding_format_is_deleted',
 'enc_product_code',
 'enc_advertiser',
 'encoder_group_id',
 'dish_included',
 'directv_included',
 'format_profile',
 'sfdc_advertiser_id',
 'sfdc_advertiser_name',
 'sfdc_account_id',
 'sfdc_rate_card_id',
 'parentid',
 'website',
 'industry',
 'annualrevenue',
 'numberofemployees',
 'description',
 'ownerid',
 'account_service_rep',
 'customer_type',
 'original_contract_date',
 'current_term_start',
 'evergreen',
 'contract_term_length_yrs',
 'adhoc_billing',
 'contract_expired',
 'annual_price_increase_per_contract',
 'price_increase_last_applied',
 'contract_missing',
 'account_notes_questions',
 'inactive_customer',
 'inactive_customer_date',
 'renewal_inactive_notes',
 'renewal_in_next_90_days',
 'delivered_to_ftp',
 'net_app_access',
 'encoder',
 'currency',
 'brand_id',
 'group_by_device',
 'encoder_id',
 'maxio_customer',
 'purchase_order_required',
 'quarterly_bulk_changes',
 'net_terms',
 'contract_type',
 'po_req',
 'sfdc_account_name']

sfdc_rate_card_cols = [
 'sfdc_rate_card_id',
 'isdeleted',
 'rate_card_name',
 'recordtypeid',
 'createddate',
 'createdbyid',
 'lastmodifieddate',
 'lastmodifiedbyid',
 'systemmodstamp',
 'tier_2_rate',
 'tier_2_start',
 'tier_1_rate',
 'tier_4_rate',
 'tier_5_start',
 'monthly_minmax',
 'tier_5_rate',
 'tier_5_label',
 'tier_3_start',
 'business_type',
 'tier_2_label',
 'cable_rate',
 'tier_3_rate',
 'currency',
 'spot_rate',
 'tier_4_label',
 'billing_type',
 'tier_1_label',
 'canadian_tv_rate',
 'rate_card_type',
 'broadcast_network_rate',
 'usage_included_in_minmax',
 'description',
 'hispanic_tv_rate',
 'tier_1_start',
 'media',
 'tier_4_start',
 'tier_3_label',
 'default_rate_card',
 'min_max_label',
 'min_max_type',
 'account',
 'bvs_name',
 'pricing_dict',
 'direct_tv_dish',
 'min_max_applied_at',
 'mediaocean_formatting',
 'sfdc_product_id',
 'sfdc_productcode',
 'start_date',
 'diginet_handling',
 'no_spot',
 'rate_currency',
 'always_bill_min',
 'no_of_tiers',
 'notes',
 'onboarding_fee',
 'seasonal_only',
 'expired',
 'detection_count_by']

In [ ]:

sfdc_account_df.columns.tolist()
for col in sfdc_advertiser_df.columns:
    new_col = col.replace('__c', '').lower()
    sfdc_advertiser_df.rename(columns={col: new_col}, inplace=True)
    if new_col == 'id':
        sfdc_advertiser_df.rename(columns={new_col: 'sfdc_advertiser_id'}, inplace=True)
    if new_col == 'name':
        sfdc_advertiser_df.rename(columns={new_col: 'sfdc_advertiser_name'}, inplace=True)
    if new_col == 'account':
        sfdc_advertiser_df.rename(columns={new_col: 'sfdc_account_id'}, inplace=True)
    if new_col == 'related_rate_card':
        sfdc_advertiser_df.rename(columns={new_col: 'sfdc_rate_card_id'}, inplace=True)
sfdc_advertiser_df.columns.tolist()
for col in sfdc_account_df.columns:
    new_col = col.replace('__c', '').lower()
    sfdc_account_df.rename(columns={col: new_col}, inplace=True)
    if new_col == 'id':
        sfdc_account_df.rename(columns={new_col: 'sfdc_account_id'}, inplace=True)
    if new_col == 'name':
        sfdc_account_df.rename(columns={new_col: 'sfdc_account_name'}, inplace=True)
sfdc_account_df.columns.tolist()
for col in sfdc_rate_card_df.columns:
    new_col = col.replace('__c', '').lower()
    sfdc_rate_card_df.rename(columns={col: new_col}, inplace=True)
    if new_col == 'id':
        sfdc_rate_card_df.rename(columns={new_col: 'sfdc_rate_card_id'}, inplace=True)
    if new_col == 'name':
        sfdc_rate_card_df.rename(columns={new_col: 'rate_card_name'}, inplace=True)
sfdc_rate_card_df.columns.tolist()
# sfdc_account_df['sfdc_account_id'] = sfdc_account_df['id']
# sfdc_account_df['sfdc_account_name'] = sfdc_account_df['name']
sfdc_account_df.sort_values(by='sfdc_account_id', inplace=True)
# sfdc_advertiser_df['sfdc_advertiser_id'] = sfdc_advertiser_df['id']
# sfdc_advertiser_df['sfdc_advertiser_name'] = sfdc_advertiser_df['name']
# sfdc_advertiser_df['sfdc_account_id'] = sfdc_advertiser_df['account']

sfdc_advertiser_df.sort_values(by=['sfdc_account_id', 'sfdc_advertiser_id', 'encoding_format_id', 'encoding_advertiser', 'encoding_product_code', 'encoding_module_code'], inplace=True)
# # sfdc_account_df[['sfdc_account_id', 'sfdc_account_name']] 
sfdc_adv_account_df = sfdc_advertiser_df.merge(sfdc_account_df, how='left', on='sfdc_account_id', suffixes=('_adv', '_acc'))
sfdc_adv_account_df = sfdc_adv_account_df[sfdc_adv_account_cols].copy()
sfdc_rate_card_df = sfdc_rate_card_df[sfdc_rate_card_cols].copy()
sfdc_adv_account_rate_card_df = sfdc_adv_account_df[sfdc_adv_account_cols].merge(sfdc_rate_card_df[sfdc_rate_card_cols], how='left', on='sfdc_rate_card_id', suffixes=('_adv', '_rc')).copy().reset_index(drop=True)


In [ ]:
attributes_list = ['product_code', 'product_name', 'donovan_agency_product_code', 'description', 'isci', 'project_name', 'advertiser', 'client_code',
                   'cable_estimate', 'spot_estimate', 'campaign', 'audience', 'audience2', 'category', 'comercial_id', 'contour_id', 'creative_offer',
                   'donovan_agency_advertiser_code', 'donovan_agency_estimate_code', 'eid', 'group', 'hd_sd', 'id', 'length', 'lob', 'media_type',
                   'message', 'misc', 'module_code', 'offer', 'offer_2', 'phone_number', 'quality', 'revision', 'show_name', 'slug', 'sport_id',
                   'sport_show_sub_category', 'spot_name', 'tag', 'text', 'title', 'veil_id', 'version_name', 'year']
for attr in attributes_list:
    encodings_df[attr] = encodings_df['attributes'].apply(lambda x: x.get(attr))
# encodings_df['product_code'] = encodings_df['attributes'].apply(lambda x: x.get('product_code'))

# Define conditions
# Ensure 'description' column does not contain None values
encodings_df['description'] = encodings_df['description'].fillna('')

# Define conditions
conditions = [
    encodings_df['product_code'].notnull(),
    encodings_df['product_code'].isnull() & encodings_df['product_name'].notnull(),
    encodings_df['product_code'].isnull() & encodings_df['product_name'].isnull() & encodings_df['donovan_agency_product_code'].notnull(),
    encodings_df['product_code'].isnull() & encodings_df['product_name'].isnull() & encodings_df['donovan_agency_product_code'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & ~encodings_df['description'].str.startswith(('TV', 'RA')),
    encodings_df['product_code'].isnull() & encodings_df['product_name'].isnull() & encodings_df['donovan_agency_product_code'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & encodings_df['description'].str.startswith(('TV', 'RA'))
]

# Define corresponding values
choices = [
    encodings_df['product_code'],
    encodings_df['product_name'],
    encodings_df['donovan_agency_product_code'],
    encodings_df['description'].str[26:30].str.strip(),
    encodings_df['description'].str[6:10].str.strip()
]

# Apply conditions and choices to create the new column
encodings_df['product_code'] = np.select(conditions, choices, default=None)



# Define conditions
conditions = [
    encodings_df['isci'].notnull(),
    encodings_df['isci'].isnull() & encodings_df['project_name'].notnull(),
    encodings_df['isci'].isnull() & encodings_df['project_name'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & ~encodings_df['description'].str.startswith(('TV', 'RA')),
    encodings_df['isci'].isnull() & encodings_df['project_name'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & encodings_df['description'].str.startswith(('TV', 'RA'))
]

# Define corresponding values
choices = [
    encodings_df['isci'],
    encodings_df['project_name'],
    encodings_df['description'].str[8:18].str.strip(),
    encodings_df['description'].str[18:38].str.strip()
]

# Apply conditions and choices to create the new column
encodings_df['isci'] = np.select(conditions, choices, default=None)


# Define conditions
conditions = [
    encodings_df['advertiser'].notnull(),
    encodings_df['advertiser'].isnull() & encodings_df['client_code'].notnull(),
    encodings_df['advertiser'].isnull() & encodings_df['client_code'].isnull() & encodings_df['donovan_agency_advertiser_code'].notnull(),
    encodings_df['advertiser'].isnull() & encodings_df['project_name'].isnull() & encodings_df['donovan_agency_advertiser_code'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & ~encodings_df['description'].str.startswith(('TV', 'RA')),
    encodings_df['advertiser'].isnull() & encodings_df['project_name'].isnull() & encodings_df['donovan_agency_advertiser_code'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & encodings_df['description'].str.startswith(('TV', 'RA'))
]

# Define corresponding values
choices = [
    encodings_df['advertiser'],
    encodings_df['client_code'],
    encodings_df['donovan_agency_advertiser_code'],
    encodings_df['description'].str[22:26].str.strip(),
    encodings_df['description'].str[2:6].str.strip()
]

# Apply conditions and choices to create the new column
encodings_df['advertiser'] = np.select(conditions, choices, default=None)

#  new cell

# sfdc_account_df['sfdc_account_id'] = sfdc_account_df['Id']
# sfdc_account_df['sfdc_account_name'] = sfdc_account_df['Name']
sfdc_bvs_customer_df['customer_id'] = sfdc_bvs_customer_df['customer_id__c'].astype(int)
sfdc_bvs_customer_df['sfdc_account_id'] = sfdc_bvs_customer_df['Account__c']
account_cols = ['sfdc_account_id', 'sfdc_account_name']
customer_cols = ['customer_id', 'sfdc_account_id']
sfdc_cust_account_df = sfdc_account_df[account_cols].merge(sfdc_bvs_customer_df[customer_cols], on='sfdc_account_id', how='inner', suffixes=('_sfdc_account', '_sfdc_customer'))
# sfdc_cust_account_df['customer_id'] = sfdc_cust_account_df['customer_id__c'].astype(int)
sfdc_cust_account_df
sfdc_bvs_cust_account_df = sfdc_cust_account_df.merge(customers_df, left_on='customer_id', right_on='customer_id', how='inner', suffixes=('_sfdc', '_avs'))

# new cell

core_functions.rename_columns(encoder_groups_df, 'encoder_group_')
core_functions.rename_columns(encoders_df, 'encoder_')
core_functions.rename_columns(formats_df, 'format_')
core_functions.rename_columns(customers_df, 'customer_')
core_functions.rename_columns(profiles_df, 'profile_')
core_functions.rename_columns(aeismaps_df, 'aeis_')

encodings_df['encoding_id'] = encodings_df['encoding_id'].astype(int)
encoders_groups_df = encoders_df.merge(encoder_groups_df, left_on='encoder_group_id', right_on='encoder_group_id', how='left')
encoders_groups_df


encodings_encoders_df = encodings_df.merge(encoders_groups_df, left_on='encoder_id', right_on='encoder_id', how='left', suffixes=('', '_dupe'))
encodings_encoders_df

aeismaps_df.sort_values(by=['aeis__encoding_id', 'aeis_id', ], inplace=True)
aeismaps_df = aeismaps_df.loc[(aeismaps_df['aeis_id'].fillna(0.0).astype(int) > 0) & (aeismaps_df['aeis__encoding_id'].fillna(0.0).astype(int) > 0)]
aeismaps_df.drop_duplicates(subset=('aeis__encoding_id' ), inplace=True)
aeismaps_df

encodings_aeis_df = encodings_encoders_df.merge(aeismaps_df, left_on='encoding_id', right_on='aeis__encoding_id', how='left')
encodings_aeis_df

formats_df['format__customer_id'] = formats_df['format__customer_id'].astype(int)
formats_df['format__profile_id'] = formats_df['format__profile_id'].astype(int)
sfdc_bvs_cust_account_df['customer_id'] = sfdc_bvs_cust_account_df['customer_id'].astype(int)
formats_customers_df = formats_df.merge(sfdc_bvs_cust_account_df, left_on='format__customer_id', right_on='customer_id', how='left')
formats_customers_df

formats_customers_profiles_df = formats_customers_df.merge(profiles_df, left_on='format__profile_id', right_on='profile_id', how='left')
formats_customers_profiles_df

encodings_bvs_df = encodings_aeis_df.merge(formats_customers_profiles_df, on='format_id',  how='left', suffixes=('', '_drop'))
for col in encodings_bvs_df.columns:
    if col.endswith('_drop'):
        encodings_bvs_df.drop(columns=col, inplace=True)
encodings_bvs_df

encodings_bvs_df['ad_prod_campaign'] = None
encodings_bvs_df['advertiser'] = encodings_bvs_df['advertiser'].fillna('')
encodings_bvs_df['product_code'] = encodings_bvs_df['product_code'].fillna('')
encodings_bvs_df['campaign'] = encodings_bvs_df['campaign'].fillna('')

# Update ad_prod_campaign
encodings_bvs_df['ad_prod_campaign'] = encodings_bvs_df.apply(
    lambda row: f"{row['advertiser'].strip()}-{row['product_code'].strip()}-{row['campaign'].strip()}".replace(' ', '_') if pd.isnull(row['ad_prod_campaign']) else row['ad_prod_campaign'],
    axis=1
)

# new cell

encodings_bvs_df.sort_values(by=['sfdc_account_id', 'format_id', 'profile_id', 'customer_id', 'encoding_id'], inplace=True)

# new cell




In [ ]:
# Sample DataFrames
# encodings_bvs_df = pd.DataFrame(...)
# sfdc_advertiser_df = pd.DataFrame(...)




# # Define a function to perform the updates
# def update_encodings_bvs_df(encodings_bvs_df, sfdc_advertiser_df):
#     # First update
#     mask = encodings_bvs_df['ad_prod_campaign'].isnull()
#     encodings_bvs_df.loc[mask, 'ad_prod_campaign'] = encodings_bvs_df[mask].apply(
#         lambda row: f"{row['advertiser'].strip()}-{row['product_code'].strip()}-{row['campaign'].strip()}".replace(' ', '_'),
#         axis=1
#     )

#     # Define a function to perform the updates based on conditions
#     def update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, left_on, right_on, match_type):
#         merged_df = encodings_bvs_df.merge(sfdc_advertiser_df, how='left', left_on=left_on, right_on=right_on)
#         mask = (merged_df['sfdc_advertiser_id'].isnull()) & (merged_df['Encoding_Format_ID__c'].notnull()) & (merged_df['format_id'] == merged_df['Encoding_Format_ID__c']) & (merged_df['Match_Type__c'] == match_type)
#         encodings_bvs_df.loc[mask, 'sfdc_advertiser_id'] = merged_df.loc[mask, 'Id']
#         encodings_bvs_df.loc[mask, 'sfdc_advertiser_name'] = merged_df.loc[mask, 'Name']
#         encodings_bvs_df.loc[mask, 'sfdc_advertiser_match_type'] = merged_df.loc[mask, 'Match_Type__c']
#         encodings_bvs_df.loc[mask, 'sfdc_rate_card_id'] = merged_df.loc[mask, 'Related_Rate_Card__c']
#         encodings_bvs_df.loc[mask, 'advertiser_updated'] = pd.Timestamp.now()

#     # Apply updates based on different conditions
#     update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, 'product_code', 'Enc_Product_Code__c', 'encoding_product_code_multiple')
#     update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, 'product_code', 'Enc_Product_Code__c', 'encoding_product_code')
#     update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, 'product_code', 'Enc_Product_Code__c', 'encoding_product_code_ignore_format')
#     update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, 'advertiser', 'Enc_Advertiser__c', 'encoding_advertiser')
#     update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, 'advertiser', 'Enc_Advertiser__c', 'encoding_advertiser_ignore_format')
#     update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, 'format_id', 'Encoding_Format_ID__c', 'encoding_format')

# # Call the function to update the DataFrame
# update_encodings_bvs_df(encodings_bvs_df, sfdc_advertiser_df)

# print(encodings_bvs_df)

In [ ]:
print(f"Starting with {len(encodings_bvs_df)} rows")
accounted_for = 0
print(f"Accounted for {accounted_for} rows")

mask = sfdc_advertiser_df['match_type'] == 'encoding_product_code_multiple'
print(len(sfdc_advertiser_df.loc[mask]))
encoding_product_code_multiple_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()
accounted_for += len(encoding_product_code_multiple_sfdc_advertiser_df)
print(f"Accounted for {accounted_for} rows")

mask = sfdc_advertiser_df['match_type'] == 'encoding_product_code'
print(len(sfdc_advertiser_df.loc[mask]))
encoding_product_code_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()
accounted_for += len(encoding_product_code_sfdc_advertiser_df)
print(f"Accounted for {accounted_for} rows")

mask = sfdc_advertiser_df['match_type'] == 'encoding_advertiser'
print(len(sfdc_advertiser_df.loc[mask]))
encoding_advertiser_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()
accounted_for += len(encoding_advertiser_sfdc_advertiser_df)
print(f"Accounted for {accounted_for} rows")




mask = sfdc_advertiser_df['match_type'] == 'encoding_advertiser_ignore_format'
print(len(sfdc_advertiser_df.loc[mask]))
encoding_advertiser_ignore_format_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()
accounted_for += len(encoding_advertiser_ignore_format_sfdc_advertiser_df)
print(f"Accounted for {accounted_for} rows")


mask = sfdc_advertiser_df['match_type'] == 'encoding_format'
print(len(sfdc_advertiser_df.loc[mask]))
encoding_format_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()
accounted_for += len(encoding_format_sfdc_advertiser_df)
print(f"Accounted for {accounted_for} rows")
print("finished normal matches")
# 
# 
# 

mask = sfdc_advertiser_df['match_type'] == 'encoder_group'
print(len(sfdc_advertiser_df.loc[mask]))
encoder_group_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()
accounted_for += len(encoder_group_sfdc_advertiser_df)
print(f"Accounted for {accounted_for} rows")

mask = sfdc_advertiser_df['match_type'] == 'Clone'
print(len(sfdc_advertiser_df.loc[mask]))
clone_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()
accounted_for += len(clone_sfdc_advertiser_df)
print(f"Accounted for {accounted_for} rows")



mask = sfdc_advertiser_df['match_type'].isna()
print(len(sfdc_advertiser_df.loc[mask]))
null_match_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()
accounted_for += len(null_match_sfdc_advertiser_df)
print(f"Accounted for {accounted_for} rows")
print(f"Started with {len(sfdc_advertiser_df)} rows and accounted for {accounted_for} rows")




# array(['encoding_format', 'encoder_group', 'Clone',
    #    'encoding_product_code_multiple', None, 'encoding_advertiser',
    #    'encoding_product_code', 'encoding_advertiser_ignore_format'],
    #   dtype=object)
# 14327 total
# encoding_format: 14174
# encoder_group: 32
# Clone: 2
# encoding_product_code_multiple: 17
# encoding_advertiser: 7
# encoding_product_code: 37
# encoding_advertiser_ignore_format: 45
#  null: 13
print(14327 - 14174 - 32 - 2 - 17 - 7 - 37 - 45 - 13)
sfdc_advertiser_df.loc[mask]

In [ ]:
# profiles_df.loc[profiles_df['profile__default_asset_code'] != 'isci']

# expanded_df = profiles_df['profile__attributes'].apply(pd.Series)

# # Combine the original DataFrame with the expanded DataFrame
# result_df = pd.concat([profiles_df, expanded_df], axis=1)
# core_functions.show_more_dataframe()
# result_df
        # CASE WHEN (e.attributes.product_code is NOT NULL) THEN e.attributes.product_code
        # WHEN (e.attributes.product_code is NULL AND e.attributes.product_name is NOT NULL) THEN e.attributes.product_name
        # WHEN (e.attributes.product_code is NULL AND e.attributes.product_name is NULL AND e.attributes.donovan_agency_product_code is NOT NULL) THEN e.attributes.donovan_agency_product_code
        # WHEN (e.attributes.product_code is NULL AND e.attributes.product_name is NULL AND e.attributes.donovan_agency_product_code is NULL and e.attributes.description is not null and length(e.attributes.description) > 10 and (e.attributes.description not like 'TV%' or e.attributes.description not like 'RA%')) THEN trim(substring(e.attributes.description, 27,4))
        # WHEN (e.attributes.product_code is NULL AND e.attributes.product_name is NULL AND e.attributes.donovan_agency_product_code is NULL and e.attributes.description is not null and length(e.attributes.description) > 10 and (e.attributes.description  like 'TV%' or e.attributes.description  like 'RA%')) THEN trim(substring(e.attributes.description, 7,4))
        # ELSE NULL END AS product_code,


In [ ]:
encodings_bvs_encoding_product_code_multiple_sfdc_advertiser_df = encodings_bvs_df.merge(encoding_product_code_multiple_sfdc_advertiser_df, left_on='format_id', right_on='encoding_format_id', how='left', suffixes=('', '_encoding_format')).dropna(subset=['sfdc_advertiser_id'])
print(f"Starting with {len(encodings_bvs_df)} rows")
print(f"Accounted for {len(encoding_product_code_multiple_sfdc_advertiser_df)} rows")
# ["ALDR", "ALGM", "ALHP"]

# new cell

encoding_product_code_multiple_sfdc_advertiser_df['product_code_list_list'] = encoding_product_code_multiple_sfdc_advertiser_df['product_code_list'].apply(lambda x: x.split(','))

# new cell

# Filter the main DataFrame to include only relevant sfdc_account_ids
meta_df = encoding_product_code_multiple_sfdc_advertiser_df
sfdc_account_ids = meta_df['sfdc_account_id'].unique().tolist()
working_df = encodings_bvs_df[encodings_bvs_df['sfdc_account_id'].isin(sfdc_account_ids)].copy()

# Ensure 'product_code' column is filled with empty strings for null values
working_df['product_code'].fillna('', inplace=True)
# working_df.drop(columns=['sfdc_advertiser_id'], inplace=True)
if 'sfdc_advertiser_id' in working_df.columns:
    working_df.drop(columns=['sfdc_advertiser_id'], inplace=True)

# Explode product_code_list_list into multiple rows
encoding_expanded_df = meta_df.copy()
encoding_expanded_df = encoding_expanded_df.explode('product_code_list_list')
encoding_expanded_df['product_code_list_list'] = encoding_expanded_df['product_code_list_list'].str.replace('"', '').copy()
encoding_expanded_df['encoding_format_id'] = encoding_expanded_df['encoding_format_id'].astype(int)
encoding_expanded_df[['product_code_list_list', 'encoding_format_id', 'sfdc_advertiser_id']]
# encoding_expanded_master_df = encoding_expanded_df.sort_values(by=['encoding_format_id', 'product_code_list_list'], inplace=True)
encoding_expanded_df.dropna(subset=['sfdc_advertiser_id'], inplace=True)
encoding_expanded_df.drop_duplicates(subset=['encoding_format_id', 'product_code_list_list'], keep='first', inplace=True)
# encoding_expanded_master_df['encoding_format_id'].fillna('0', inplace=True)
# encoding_expanded_master_df['product_code_list_list'].fillna('', inplace=True)
# encoding_expanded_master2_df = encoding_expanded_master_df.drop_duplicates(subset=['encoding_format_id', 'product_code_list_list'], keep='first')
# len(encoding_expanded_master2_df)
working_df['format_id'] = working_df['format_id'].astype(int)
# Perform the merge based on format_id and product_code
merged_df = working_df.merge(
    encoding_expanded_df,
    left_on=['format_id', 'product_code'],
    right_on=['encoding_format_id', 'product_code_list_list'],
    how='left', suffixes=('', '_encoding_format')
)
merged_df = merged_df.dropna(subset=['sfdc_advertiser_id'])
merged_df['encoding_id'] = merged_df['encoding_id'].astype(int)
working_df['encoding_id'] = working_df['encoding_id'].astype(int)
merged_df[['encoding_id','sfdc_account_id']]
print(f"Starting with {len(working_df)} rows")
working_df2 = working_df.merge(merged_df[['encoding_id', 'sfdc_advertiser_id']], on='encoding_id', how='left',)
print(f"Accounted for {len(working_df2)} rows")
working_df2.dropna(subset=['sfdc_advertiser_id'], inplace=True)
# # Assign the advertiser ID to the original DataFrame
# working_df['sfdc_advertiser_id'] = merged_df['sfdc_advertiser_id']
# # working_df.dropna(subset=['sfdc_advertiser_id'])
# working_df
working_df2.columns.tolist()

new_encodings_bvs_df = pd.DataFrame(columns=working_df2.columns)
new_encodings_bvs_df = pd.concat([new_encodings_bvs_df, working_df2], ignore_index=True)
new_encodings_encodings_ids = new_encodings_bvs_df['encoding_id'].unique().tolist()
len(new_encodings_bvs_df)

# new cell

meta_df = encoding_product_code_sfdc_advertiser_df
sfdc_account_ids = meta_df['sfdc_account_id'].unique().tolist()
working_df = encodings_bvs_df[(encodings_bvs_df['sfdc_account_id'].isin(sfdc_account_ids)) & ~(encodings_bvs_df['encoding_id'].isin(new_encodings_encodings_ids))].copy()

# Ensure 'product_code' column is filled with empty strings for null values
working_df['product_code'].fillna('', inplace=True)
if 'sfdc_advertiser_id' in working_df.columns:
    working_df.drop(columns=['sfdc_advertiser_id'], inplace=True)
# working_df.drop(columns=['sfdc_advertiser_id'], inplace=True)
encoding_expanded_df = meta_df.copy()

encoding_expanded_df[['sfdc_account_id', 'sfdc_advertiser_name', 'sfdc_advertiser_id', 'encoding_format_id', 'encoding_product_code', 'product_code']]
encoding_expanded_df.rename(columns={'encoding_format_id': 'format_id'}, inplace=True)
encoding_expanded_df.drop(columns=['product_code'], inplace=True)
encoding_expanded_df.rename(columns={'enc_product_code': 'product_code'}, inplace=True)
encoding_expanded_df
encoding_expanded_df = encoding_expanded_df.dropna(subset=['sfdc_advertiser_id']).copy()
encoding_expanded_slim_df = encoding_expanded_df[['sfdc_account_id', 'sfdc_advertiser_id', 'format_id', 'product_code']].copy()
encoding_expanded_slim_df

merged_df = working_df.merge(encoding_expanded_slim_df, on=['sfdc_account_id', 'format_id', 'product_code'], how='left')
merged_df.dropna(subset=['sfdc_advertiser_id'], inplace=True)
merged_df
new_encodings_bvs_df
new_encodings_bvs_df = pd.concat([new_encodings_bvs_df, merged_df], ignore_index=True)
new_encodings_encodings_ids = new_encodings_bvs_df['encoding_id'].unique().tolist()
len(new_encodings_bvs_df)

# new cell

# encoding_product_code_ignore_format
meta_df =encoding_advertiser_ignore_format_sfdc_advertiser_df
sfdc_account_ids = meta_df['sfdc_account_id'].unique().tolist()
working_df = encodings_bvs_df[(encodings_bvs_df['sfdc_account_id'].isin(sfdc_account_ids)) & ~(encodings_bvs_df['encoding_id'].isin(new_encodings_encodings_ids))].copy()

# Ensure 'product_code' column is filled with empty strings for null values
working_df['product_code'].fillna('', inplace=True)
if 'sfdc_advertiser_id' in working_df.columns:
    working_df.drop(columns=['sfdc_advertiser_id'], inplace=True)
encoding_expanded_df = meta_df.copy()

encoding_expanded_df[['sfdc_account_id', 'sfdc_advertiser_name', 'sfdc_advertiser_id',  'encoding_product_code', 'product_code']]
# encoding_expanded_df.rename(columns={'encoding_format_id': 'format_id'}, inplace=True)
encoding_expanded_df.drop(columns=['product_code'], inplace=True)
encoding_expanded_df.rename(columns={'enc_product_code': 'product_code'}, inplace=True)
encoding_expanded_df
encoding_expanded_df = encoding_expanded_df.dropna(subset=['sfdc_advertiser_id']).copy()
encoding_expanded_slim_df = encoding_expanded_df[['sfdc_account_id', 'sfdc_advertiser_id',  'product_code']].copy()
encoding_expanded_slim_df

merged_df = working_df.merge(encoding_expanded_slim_df, on=['sfdc_account_id',  'product_code'], how='left')
merged_df.dropna(subset=['sfdc_advertiser_id'], inplace=True)
merged_df
new_encodings_bvs_df
new_encodings_bvs_df = pd.concat([new_encodings_bvs_df, merged_df], ignore_index=True)
new_encodings_encodings_ids = new_encodings_bvs_df['encoding_id'].unique().tolist()
len(new_encodings_bvs_df)

# new cell

# encoding_advertiser
meta_df = encoding_advertiser_sfdc_advertiser_df
sfdc_account_ids = meta_df['sfdc_account_id'].unique().tolist()
working_df = encodings_bvs_df[(encodings_bvs_df['sfdc_account_id'].isin(sfdc_account_ids)) & ~(encodings_bvs_df['encoding_id'].isin(new_encodings_encodings_ids))].copy()

# Ensure 'product_code' column is filled with empty strings for null values
working_df['advertiser'].fillna('', inplace=True)
if 'sfdc_advertiser_id' in working_df.columns:
    working_df.drop(columns=['sfdc_advertiser_id'], inplace=True)
encoding_expanded_df = meta_df.copy()

encoding_expanded_df[['sfdc_account_id', 'sfdc_advertiser_name', 'sfdc_advertiser_id', 'encoding_format_id', 'enc_advertiser']]
encoding_expanded_df.rename(columns={'encoding_format_id': 'format_id'}, inplace=True)
encoding_expanded_df.drop(columns=['product_code'], inplace=True)
encoding_expanded_df.rename(columns={'enc_advertiser': 'advertiser'}, inplace=True)
encoding_expanded_df
encoding_expanded_df = encoding_expanded_df.dropna(subset=['sfdc_advertiser_id']).copy()
encoding_expanded_slim_df = encoding_expanded_df[['sfdc_account_id', 'sfdc_advertiser_id', 'format_id', 'advertiser']].copy()
encoding_expanded_slim_df

merged_df = working_df.merge(encoding_expanded_slim_df, on=['sfdc_account_id', 'format_id', 'advertiser'], how='left')
merged_df.dropna(subset=['sfdc_advertiser_id'], inplace=True)
merged_df
new_encodings_bvs_df
new_encodings_bvs_df = pd.concat([new_encodings_bvs_df, merged_df], ignore_index=True)
new_encodings_encodings_ids = new_encodings_bvs_df['encoding_id'].unique().tolist()
len(new_encodings_bvs_df)

# new cell

# encoding_advertiser_ignore_format
meta_df = encoding_advertiser_ignore_format_sfdc_advertiser_df
sfdc_account_ids = meta_df['sfdc_account_id'].unique().tolist()
working_df = encodings_bvs_df[(encodings_bvs_df['sfdc_account_id'].isin(sfdc_account_ids)) & ~(encodings_bvs_df['encoding_id'].isin(new_encodings_encodings_ids))].copy()

# Ensure 'product_code' column is filled with empty strings for null values
working_df['advertiser'].fillna('', inplace=True)
if 'sfdc_advertiser_id' in working_df.columns:
    working_df.drop(columns=['sfdc_advertiser_id'], inplace=True)
encoding_expanded_df = meta_df.copy()

encoding_expanded_df[['sfdc_account_id', 'sfdc_advertiser_name', 'sfdc_advertiser_id',  'enc_advertiser']]
# encoding_expanded_df.rename(columns={'encoding_format_id': 'format_id'}, inplace=True)
encoding_expanded_df.drop(columns=['product_code'], inplace=True)
encoding_expanded_df.rename(columns={'enc_advertiser': 'advertiser'}, inplace=True)
encoding_expanded_df
encoding_expanded_df = encoding_expanded_df.dropna(subset=['sfdc_advertiser_id']).copy()
encoding_expanded_slim_df = encoding_expanded_df[['sfdc_account_id', 'sfdc_advertiser_id',  'advertiser']].copy()
encoding_expanded_slim_df

merged_df = working_df.merge(encoding_expanded_slim_df, on=['sfdc_account_id', 'advertiser'], how='left')
merged_df.dropna(subset=['sfdc_advertiser_id'], inplace=True)
merged_df
new_encodings_bvs_df
new_encodings_bvs_df = pd.concat([new_encodings_bvs_df, merged_df], ignore_index=True)
new_encodings_encodings_ids = new_encodings_bvs_df['encoding_id'].unique().tolist()
len(new_encodings_bvs_df)


# new cell

# encoding_format
meta_df = encoding_format_sfdc_advertiser_df
sfdc_account_ids = meta_df['sfdc_account_id'].unique().tolist()
working_df = encodings_bvs_df[(encodings_bvs_df['sfdc_account_id'].isin(sfdc_account_ids)) & ~(encodings_bvs_df['encoding_id'].isin(new_encodings_encodings_ids))].copy()

# Ensure 'product_code' column is filled with empty strings for null values
working_df['advertiser'].fillna('', inplace=True)
if 'sfdc_advertiser_id' in working_df.columns:
    working_df.drop(columns=['sfdc_advertiser_id'], inplace=True)
encoding_expanded_df = meta_df.copy()

encoding_expanded_df[['sfdc_account_id', 'sfdc_advertiser_name', 'sfdc_advertiser_id', 'encoding_format_id']]
encoding_expanded_df.rename(columns={'encoding_format_id': 'format_id'}, inplace=True)
# encoding_expanded_df.drop(columns=['product_code'], inplace=True)
# encoding_expanded_df.rename(columns={'enc_advertiser': 'advertiser'}, inplace=True)
encoding_expanded_df
encoding_expanded_df = encoding_expanded_df.dropna(subset=['sfdc_advertiser_id']).copy()
encoding_expanded_slim_df = encoding_expanded_df[['sfdc_account_id', 'sfdc_advertiser_id', 'format_id']].copy()
encoding_expanded_slim_df

merged_df = working_df.merge(encoding_expanded_slim_df, on=['sfdc_account_id', 'format_id'], how='left')
merged_df.dropna(subset=['sfdc_advertiser_id'], inplace=True)
merged_df
new_encodings_bvs_df
new_encodings_bvs_df = pd.concat([new_encodings_bvs_df, merged_df], ignore_index=True)
new_encodings_encodings_ids = new_encodings_bvs_df['encoding_id'].unique().tolist()
len(new_encodings_bvs_df)
# 1686537

# new cell

new_encodings_bvs_df.sort_values(by=['sfdc_account_id', 'sfdc_advertiser_id','format_id', 'profile_id', 'customer_id', 'encoding_id'], inplace=True)

# new cell

# new_encodings_bvs_merge_df = new_encodings_bvs_df[['encoding_id', 'sfdc_advertiser_id']].drop_
# new_encodings_bvs_merge_df
# encodings_bvs_df_to_write = encodings_bvs_df.merge(new_encodings_bvs_merge_df, on='encoding_id', how='left')
# encodings_bvs_df_to_write
len(encodings_bvs_df)
len(encodings_bvs_df[~(encodings_bvs_df['encoding_id'].isin(new_encodings_encodings_ids))]) + len(new_encodings_bvs_df)
encodings_bvs_df['sfdc_advertiser_id'] = ''
# 1686537
# 1686866
encodings_bvs_df_to_write = pd.concat([encodings_bvs_df[~(encodings_bvs_df['encoding_id'].isin(new_encodings_encodings_ids))], new_encodings_bvs_df], ignore_index=True).reset_index(drop=True)

# new cell

# encodings_bvs_df_to_write.sort_values(by=['sfdc_account_id'], inplace=True)
billing_last_updated = pd.Timestamp.now()
encodings_bvs_df_to_write['billing_last_updated'] = billing_last_updated
billing_last_audit_id = core_functions.generate_uuid()
encodings_bvs_df_to_write['billing_last_audit_id'] = billing_last_audit_id
# encodings_bvs_df_to_write

# new cell

pd.options.display.max_rows = None
int_cols = ['encoding_id','encoder_group_id']
# encodings_bvs_df_to_write[]
# print(encodings_bvs_df_to_write.dtypes)
# encodings_bvs_df_to_write['aeis_id'].isna()
mask = encodings_bvs_df_to_write['aeis_id'].isna()
encodings_bvs_df_to_write[mask,[['encoding_id','encoded_timestamp']]]

# new cell

import json
from datetime import datetime

process_df = encodings_bvs_df_to_write[0:10000].copy()

nested_columns = ['attributes']
for col in nested_columns:
    process_df[col] = process_df[col].apply(lambda x: json.dumps(x) if pd.notna(x) else '')
    
def parse_iso_with_timezone(ts):
    # Replace timezone colon
    ts = ts.replace(":", "", 1) if "+" in ts or "-" in ts else ts
    return datetime.strptime(ts, "%Y-%m-%dT%H:%M:%S.%f%z")

# parsed_timestamps = [parse_iso_with_timezone(ts) for ts in process_df['encoded_timestamp'] ]
process_df['encoded_timestamp'] = process_df['encoded_timestamp'].apply(parse_iso_with_timezone)

# print(parsed_timestamps)
# process_df['encoded_timestamp'] = pd.to_datetime(process_df['encoded_timestamp'])
# process_df = core_functions.convert_to_string_except_exclusions(process_df, exclude_columns=['encoded_timestamp', 'encoding_id', 'format_id','customer_id', 'profile_id', 'billing_last_updated', 'deleted', 'profile_deleted'])
# process_df.drop(columns=['attributes'], inplace=True)
veil_storage_options = config.get('VEIL_GCS_STORAGE_OPTIONS')
veil_storage_options = config.get('VEIL_GCS_STORAGE_OPTIONS')
n90_storage_options = config.get('N90_GCS_STORAGE_OPTIONS')

veil_billing_bucket = config.get('veil_billing').get('billing_gcs_bucket_id')
process_df['encoded_timestamp']
# process_df['profile__attributes']
# n90_bucket = 'n90_veil_partner'
# veil_output_prefix = 'encodings'
# n90_output_prefix = 'advocado-looker/avs_prod/encodings'
# partition_cols = ['sfdc_account_id']
# core_functions.write_hive_partitioned_parquet(process_df, veil_billing_bucket, veil_output_prefix, partition_cols, veil_storage_options)
# print(f"Finished writing to {veil_billing_bucket}/{veil_output_prefix}")
# core_functions.write_hive_partitioned_parquet(process_df, n90_bucket, n90_output_prefix, partition_cols, n90_storage_options)
# print(f"Finished writing to {n90_bucket}/{n90_output_prefix}")

# new cell

print(encodings_bvs_df_to_write.dtypes)


# new cell

veil_storage_options = config.get('VEIL_GCS_STORAGE_OPTIONS')
veil_storage_options = config.get('VEIL_GCS_STORAGE_OPTIONS')
n90_storage_options = config.get('N90_GCS_STORAGE_OPTIONS')
n90_storage_options
veil_billing_bucket = config.get('veil_billing').get('billing_gcs_bucket_id')
veil_billing_bucket
# n90_bucket = 'n90_veil_partner'
# veil_output_prefix = 'encodings'
# n90_output_prefix = 'advocado-looker/avs_prod/encodings'
# partition_cols = ['sfdc_account_id']

# core_functions.write_hive_partitioned_parquet(encodings_bvs_df_to_write, n90_bucket, n90_output_prefix, partition_cols, n90_storage_options)
# print(f"Finished writing to {n90_bucket}/{n90_output_prefix}")

# new cell

